### Experimenting with BERTScore

In [10]:
import sys
sys.path.append(r"D:\COURS\A4\S8 - ESILV\Stage\Work\Repositories\bert_score")

from bert_score.score import score as bscore

In [11]:
o = bscore(["I am Marius"], ["My name is marius"], lang="en")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Experimenting with Roberta tokenizer

In [1]:
from transformers import RobertaTokenizer

d:\COURS\A4\S8 - ESILV\Stage\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
test = tokenizer("I am Marius")

In [3]:
out = tokenizer("I am Marius")

In [20]:
out

{'input_ids': [0, 100, 524, 1127, 6125, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [21]:
test.input_ids

[0, 100, 524, 1127, 6125, 2]

### Experimenting with AutoModel class

In [4]:
from transformers import AutoModel
from transformers import RobertaTokenizer

In [5]:
model = AutoModel.from_pretrained("roberta-large")
tokenizer = RobertaTokenizer.from_pretrained("roberta-large")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
sentence = "I am Marius"
tokens = tokenizer(sentence)
token_ids = tokens["input_ids"]
masks = tokens["attention_mask"]

In [8]:
type(masks)

list

In [ ]:
out = model(token_ids, attention_mask=masks, output_hidden_states=True)

### BERT Embedding - Discovery

In [34]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states=True)
model.eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [15]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
text = "My name is Marius."
marked_text = "[CLS]"+text+"[SEP]"
tokenized_text = tokenizer.tokenize(marked_text)
print(tokenized_text)

['[CLS]', 'my', 'name', 'is', 'marius', '.', '[SEP]']


In [16]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
for tup in zip(tokenized_text, indexed_tokens):
    print('{:<12}{:>6,}'.format(tup[0], tup[1]))

[CLS]          101
my           2,026
name         2,171
is           2,003
marius      20,032
.            1,012
[SEP]          102


In [17]:
#mark tokenks as belonging to sentence 1.
segment_ids = [1]*len(tokenized_text)
print(segment_ids)

[1, 1, 1, 1, 1, 1, 1]


In [18]:
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensor = torch.tensor([segment_ids])
print(tokens_tensor)
print(segments_tensor)

tensor([[  101,  2026,  2171,  2003, 20032,  1012,   102]])
tensor([[1, 1, 1, 1, 1, 1, 1]])


In [21]:
with torch.no_grad():
    outputs = model(tokens_tensor, segments_tensor)
outputs.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [27]:
bert_hidden_states = outputs[-1]
len(bert_hidden_states)

13

In [31]:
token_embeddings = torch.stack(bert_hidden_states, dim=0)
token_embeddings.size()

torch.Size([13, 1, 7, 768])

In [32]:
#here we have 1 sentence so we remove the batch size
token_embeddings = torch.squeeze(token_embeddings, dim=1)
token_embeddings.size()

torch.Size([13, 7, 768])

In [33]:
#swap dim 0 and 1
token_embeddings = token_embeddings.permute(1, 0, 2)
token_embeddings.size()

torch.Size([7, 13, 768])

#### Word Vectors

In [47]:
#concatenate layers
token_vec_cat = []
for token in token_embeddings:
    cat_vec = torch.cat((token[-1], token[-2], token[-3], token[-4]), dim=0)
    token_vec_cat.append(cat_vec)
print("Shape:", len(token_vec_cat),"x", len(token_vec_cat[0]))

Shape: 7 x 3072


In [59]:
for i in range(len(token_vec_cat)):
    if len(token_vec_cat[i] != 3072):
        print(i)
        print(len(token_vec_cat[i]))

0
3072
1
3072
2
3072
3
3072
4
3072
5
3072
6
3072


In [78]:
test = [token.tolist() for token in token_vec_cat]
test = np.array(test)

In [79]:
test.shape

(7, 3072)

In [48]:
#sum layers
token_vec_sum = []
for token in token_embeddings:
    cat_vec = torch.sum(token[-4:], dim=0)
    token_vec_sum.append(cat_vec)
print("Shape:", len(token_vec_sum),"x", len(token_vec_sum[0]))

Shape: 7 x 768


#### Sentence Vectors

In [51]:
token_vecs = bert_hidden_states[-2][0]
sentence_embedding = torch.mean(token_vecs, dim=0)
sentence_embedding.size()

torch.Size([768])

### Vector Visualization

In [82]:
from umap import UMAP
import plotly.express as px

In [80]:
umap3D = UMAP(n_components=3, init='random', random_state=0)
proj3D = umap3D.fit_transform(test)

d:\COURS\A4\S8 - ESILV\Stage\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\umap\umap_.py:2344: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


In [85]:
f = px.scatter_3d(proj3D, x=0, y=1, z=2)
f.show()

### BERT Sentence-level Embedding - Paragraphs 

In [6]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
corpus = "Wikipedia is a multilingual, free, online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and using a wiki-based editing system called MediaWiki. Wikipedia is the largest and most-read reference work in history. It is consistently one of the 10 most popular websites ranked by Similarweb and formerly Alexa; as of 2023, Wikipedia was ranked the 5th most popular site in the world according to Semrush. It is hosted by the Wikimedia Foundation, an American non-profit organization funded mainly through donations."
delimiter="."
print(corpus)

Wikipedia is a multilingual, free, online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and using a wiki-based editing system called MediaWiki. Wikipedia is the largest and most-read reference work in history. It is consistently one of the 10 most popular websites ranked by Similarweb and formerly Alexa; as of 2023, Wikipedia was ranked the 5th most popular site in the world according to Semrush. It is hosted by the Wikimedia Foundation, an American non-profit organization funded mainly through donations.


In [20]:
splited = [sentence+delimiter for sentence in corpus.split(".")]
max_len = max(len(x) for x in splited)

input_ids = []
attention_masks = []
for sentence in splited:
    encoded = tokenizer.encode_plus(sentence, 
                                    add_special_tokens=True,
                                    max_length=max_len+1,
                                    pad_to_max_length=True,
                                    return_attention_mask=True,
                                    return_tensors='pt',
                                    truncation=True)
    input_ids.append(encoded["input_ids"])
    attention_masks.append(encoded["attention_mask"])

inputs_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

with torch.no_grad():
    output = model(inputs_ids, attention_mask=attention_masks)
hidden_state = output.last_hidden_state
cls_emb = hidden_state[:,0,:]
cls_emb = cls_emb.detach().numpy()
np.shape(cls_emb)

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the

(5, 768)

In [19]:
cls_emb

array([[-0.10181614, -0.04256969, -0.37916115, ..., -0.01706432,
         0.38435265,  0.77578974],
       [-0.15074998,  0.16900486, -0.4508034 , ..., -0.41086987,
         0.6468059 ,  0.44568998],
       [-0.00590458,  0.07766522, -0.09788167, ..., -0.05835519,
         0.6742643 ,  0.6657301 ],
       [-0.46835747,  0.30540103, -0.3483228 , ..., -0.31408456,
         0.18837832,  0.8075387 ],
       [-0.1548306 ,  0.25915366,  0.14536104, ..., -0.2707459 ,
         0.49331143,  0.22958532]], dtype=float32)

In [14]:
len(splited)

5

### BERT Word-level Embedding - Paragraphs

In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

model = BertModel.from_pretrained('bert-base-uncased',
                                  output_hidden_states=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

corpus = "Wikipedia is a multilingual, free, online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and using a wiki-based editing system called MediaWiki. Wikipedia is the largest and most-read reference work in history. It is consistently one of the 10 most popular websites ranked by Similarweb and formerly Alexa; as of 2023, Wikipedia was ranked the 5th most popular site in the world according to Semrush. It is hosted by the Wikimedia Foundation, an American non-profit organization funded mainly through donations."
delimiter="."
print(corpus)

In [21]:
splited = [sentence+delimiter for sentence in corpus.split(".")]
max_len = max(len(x) for x in splited)

input_ids = []
attention_masks = []
for sentence in splited:
    encoded = tokenizer.encode_plus(sentence, 
                                    add_special_tokens=True,
                                    max_length=max_len+1,
                                    pad_to_max_length=True,
                                    return_attention_mask=True,
                                    return_tensors='pt',
                                    truncation=True)
    input_ids.append(encoded["input_ids"])
    attention_masks.append(encoded["attention_mask"])

inputs_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

with torch.no_grad():
    output = model(inputs_ids, attention_mask=attention_masks)

In [24]:
hidden_states = output.hidden_states

In [25]:
token_embeddings = torch.stack(hidden_states, dim=0)
token_embeddings.size()

torch.Size([13, 5, 213, 768])

In [ ]:
token_embeddings.permute()

### Embedding a corpus

In [86]:
ex = "Wikipedia is a multilingual, free, online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and using a wiki-based editing system called MediaWiki. Wikipedia is the largest and most-read reference work in history. It is consistently one of the 10 most popular websites ranked by Similarweb and formerly Alexa; as of 2023, Wikipedia was ranked the 5th most popular site in the world according to Semrush. It is hosted by the Wikimedia Foundation, an American non-profit organization funded mainly through donations."
print(ex)

Wikipedia is a multilingual, free, online encyclopedia written and maintained by a community of volunteers, known as Wikipedians, through open collaboration and using a wiki-based editing system called MediaWiki. Wikipedia is the largest and most-read reference work in history. It is consistently one of the 10 most popular websites ranked by Similarweb and formerly Alexa; as of 2023, Wikipedia was ranked the 5th most popular site in the world according to Semrush. It is hosted by the Wikimedia Foundation, an American non-profit organization funded mainly through donations.


In [111]:
def tokenizeCorpus(corpus, model=BertModel.from_pretrained('bert-base-uncased'), tokenizer = BertTokenizer.from_pretrained('bert-base-uncased'), delimiter="."):
    splited = [sentence+delimiter for sentence in corpus.split(".")]
    SEPmarked = [sentence+" [SEP]" for sentence in splited[:-1]]
    marked_text = "[CLS] "+"".join(SEPmarked)[:-6]
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segment_ids = []
    i = 0
    for token in tokenized_text:
        segment_ids.append(i)
        if token == "[SEP]":
            i += 1
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensor = torch.tensor([segment_ids])
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensor)
    hidden_states = outputs[-1]
    
    
    return outputs

o = tokenizeCorpus(ex)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [112]:
o

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]]]), pooler_output=tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,